# Projet Python - TOP14 Rugby

Mathieu Roig

<a id="intro"></a>
# Introduction

Le projet est structuré de manière à guider l'utilisateur à travers les différentes étapes de l'analyse, de la collecte des données aux résultats. La section [Données](#données) détaille le processus de [Web-Scraping](#scraping), ainsi que les étapes de [Nettoyage](#nettoyage) et de [Regroupement](#regroupement) des données. Il y'aura ensuite une étape de [Visualisation](#visual) des données et leur [Analyse](#analyse). La dernière partie concerne les [Modèles prédictifs](#predict) avec d'abord la prédiction d'un champion une fois la saison régulière terminée puis dans un second temps la prédiction de matchs pour une saison vierge.

<a id="sommaire"></a>
### Sommaire
- [Données](#données)
  - [Web-Scraping](#scraping)
  - [Nettoyage](#nettoyage)
  - [Uniformisation](#uniformisation)
  - [Calendrier](#travail-sur-calendrier)
  - [Regroupement](#regroupement)
- [Statistiques Descriptives](#stats)
  - [Visualisation](#visual)
  - [Suivi des performances](#suivi-des-performances)
- [Modèle prédictif](#predict)
  - [Prédiction du champion](#classement)
    - [Regression Logistique](#regression-logistique)
    - [Random Forest](#random-forest)
    - [XGBoost](#xgboost)
  - [Prédiction par Elo](#prédiction-par-elo)
    - [Test sur la saison 24/25](#test-sur-la-saison-2425)
    - [Saison 25/26](#prédiction-sur-2526)

<a id="installation"></a>
### Installation

In [ ]:
#packages:
from io import StringIO
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import sys
import subprocess
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

#lxml
#subprocess.check_call([sys.executable, "-m", "pip", "install", "lxml"])

<a id="données"></a>
# Données

<a id="scraping"></a>
### Web-Scraping

Je récupère sur Wikipédia les données sur le championnat de France de rugby à XV de 2016 à aujourd'hui. Chaque saison a sa propre page wikipédia dédiée et la présentation des résultats peut légèrement différer en fonction de l'année.

In [ ]:
urls = [f"https://fr.wikipedia.org/wiki/Championnat_de_France_de_rugby_%C3%A0_XV_{year}-{year+1}"
    for year in range(2005, 2025)]
print(urls)

Nous utilisons donc plusieurs versions d'une même fonction data dans scrapData.py qui récupère les tableaux de présentation générale, du classement, de son évolution, les scores par match, et aussi la forme des équipes (série de victoires ou de défautes) en fonction des différentes semaines.

In [ ]:
sys.path.append(os.path.abspath('./scripts'))

les fonctions dataXXXX posent souvent problème par impossibilité d'accès à wikipedia, en cas d'erreur "list index out of range" (qui signifie que l'accès à la table a été interdit par le site) elles sont disponibles dans le dossier dataBRUT

In [ ]:
# from scrapData import data2425,data2324, data2223, data2122, data2021, data1920, data1619, data1516x1314, data1415x1213, data1112, data1011, data0910, data0709, data0607, data0506 

# tab0506 = data0506(urls[0])
# time.sleep(2)
# tab0607 = data0607(urls[1])
# time.sleep(2)
# tab0708 = data0709(urls[2])
# time.sleep(2)
# tab0809 = data0709(urls[3])
# time.sleep(2)
# tab0910 = data0910(urls[4])
# time.sleep(2)
# tab1011 = data1011(urls[5])
# time.sleep(2)
# tab1112 = data1112(urls[6])
# time.sleep(2)
# tab1213 = data1415x1213(urls[7])
# time.sleep(2)
# tab1314 = data1516x1314(urls[8])
# time.sleep(2)
# tab1415 = data1415x1213(urls[9])
# time.sleep(2)
# tab1516 = data1516x1314(urls[10])
# time.sleep(2)
# tab1617 = data1619(urls[11])
# time.sleep(2)
# tab1718 = data1619(urls[12])
# time.sleep(2)
# tab1819 = data1619(urls[13])
# time.sleep(2)
# tab1920 = data1920(urls[14])
# time.sleep(2)
# tab2021 = data2021(urls[15])
# time.sleep(2)
# tab2122 = data2122(urls[16])
# time.sleep(2)
# tab2223 = data2223(urls[17])
# time.sleep(2)
# tab2324 = data2324(urls[18])
# time.sleep(2)
# tab2425 = data2425(urls[19])

In [ ]:
tab0506=['','']
tab0506[0] = pd.read_csv("dataBRUT/0506/cla0506.csv")
tab0506[1] = pd.read_csv("dataBRUT/0506/res0506.csv")

tab0607 = ['', '']
tab0607[0] = pd.read_csv("dataBRUT/0607/cla0607.csv")
tab0607[1] = pd.read_csv("dataBRUT/0607/res0607.csv")

tab0708 = ['', '']
tab0708[0] = pd.read_csv("dataBRUT/0708/cla0708.csv")
tab0708[1] = pd.read_csv("dataBRUT/0708/res0708.csv")

tab0809 = ['', '']
tab0809[0] = pd.read_csv("dataBRUT/0809/cla0809.csv")
tab0809[1] = pd.read_csv("dataBRUT/0809/res0809.csv")

tab0910 = ['', '','']
tab0910[0] = pd.read_csv("dataBRUT/0910/pre0910.csv")
tab0910[1] = pd.read_csv("dataBRUT/0910/cla0910.csv")
tab0910[2] = pd.read_csv("dataBRUT/0910/res0910.csv")

tab1011 = ['', '','','']
tab1011[0] = pd.read_csv("dataBRUT/1011/pre1011.csv")
tab1011[1] = pd.read_csv("dataBRUT/1011/cla1011.csv")
tab1011[2] = pd.read_csv("dataBRUT/1011/res1011.csv")
tab1011[3] = pd.read_csv("dataBRUT/1011/evo1011.csv")

tab1112 = ['', '', '', '']
tab1112[0] = pd.read_csv("dataBRUT/1112/pre1112.csv")
tab1112[1] = pd.read_csv("dataBRUT/1112/cla1112.csv")
tab1112[2] = pd.read_csv("dataBRUT/1112/res1112.csv")
tab1112[3] = pd.read_csv("dataBRUT/1112/evo1112.csv")

tab1213 = ['', '', '', '']
tab1213[0] = pd.read_csv("dataBRUT/1213/pre1213.csv")
tab1213[1] = pd.read_csv("dataBRUT/1213/cla1213.csv")
tab1213[2] = pd.read_csv("dataBRUT/1213/res1213.csv")
tab1213[3] = pd.read_csv("dataBRUT/1213/evo1213.csv")

tab1314 = ['', '', '', '']
tab1314[0] = pd.read_csv("dataBRUT/1314/pre1314.csv")
tab1314[1] = pd.read_csv("dataBRUT/1314/cla1314.csv")
tab1314[2] = pd.read_csv("dataBRUT/1314/res1314.csv")
tab1314[3] = pd.read_csv("dataBRUT/1314/evo1314.csv")

tab1415 = ['', '', '', '']
tab1415[0] = pd.read_csv("dataBRUT/1415/pre1415.csv")
tab1415[1] = pd.read_csv("dataBRUT/1415/cla1415.csv")
tab1415[2] = pd.read_csv("dataBRUT/1415/res1415.csv")
tab1415[3] = pd.read_csv("dataBRUT/1415/evo1415.csv")

tab1516 = ['', '', '', '']
tab1516[0] = pd.read_csv("dataBRUT/1516/pre1516.csv")
tab1516[1] = pd.read_csv("dataBRUT/1516/cla1516.csv")
tab1516[2] = pd.read_csv("dataBRUT/1516/res1516.csv")
tab1516[3] = pd.read_csv("dataBRUT/1516/evo1516.csv")

tab1617 = ['', '', '', '','']
tab1617[0] = pd.read_csv("dataBRUT/1617/pre1617.csv")
tab1617[1] = pd.read_csv("dataBRUT/1617/cla1617.csv")
tab1617[2] = pd.read_csv("dataBRUT/1617/res1617.csv")
tab1617[3] = pd.read_csv("dataBRUT/1617/evo1617.csv")
tab1617[4] = pd.read_csv("dataBRUT/1617/for1617.csv")

tab1718 = ['', '', '', '', '']
tab1718[0] = pd.read_csv("dataBRUT/1718/pre1718.csv")
tab1718[1] = pd.read_csv("dataBRUT/1718/cla1718.csv")
tab1718[2] = pd.read_csv("dataBRUT/1718/res1718.csv")
tab1718[3] = pd.read_csv("dataBRUT/1718/evo1718.csv")
tab1718[4] = pd.read_csv("dataBRUT/1718/for1718.csv")

tab1819 = ['', '', '', '', '']
tab1819[0] = pd.read_csv("dataBRUT/1819/pre1819.csv")
tab1819[1] = pd.read_csv("dataBRUT/1819/cla1819.csv")
tab1819[2] = pd.read_csv("dataBRUT/1819/res1819.csv")
tab1819[3] = pd.read_csv("dataBRUT/1819/evo1819.csv")
tab1819[4] = pd.read_csv("dataBRUT/1819/for1819.csv")

tab1920 = ['', '', '', '', '']
tab1920[0] = pd.read_csv("dataBRUT/1920/pre1920.csv")
tab1920[1] = pd.read_csv("dataBRUT/1920/cla1920.csv")
tab1920[2] = pd.read_csv("dataBRUT/1920/res1920.csv")
tab1920[3] = pd.read_csv("dataBRUT/1920/evo1920.csv")
tab1920[4] = pd.read_csv("dataBRUT/1920/for1920.csv")

tab2021 = ['', '', '', '', '']
tab2021[0] = pd.read_csv("dataBRUT/2021/pre2021.csv")
tab2021[1] = pd.read_csv("dataBRUT/2021/cla2021.csv")
tab2021[2] = pd.read_csv("dataBRUT/2021/res2021.csv")
tab2021[3] = pd.read_csv("dataBRUT/2021/evo2021.csv")
tab2021[4] = pd.read_csv("dataBRUT/2021/for2021.csv")

tab2122 = ['', '', '', '', '']
tab2122[0] = pd.read_csv("dataBRUT/2122/pre2122.csv")
tab2122[1] = pd.read_csv("dataBRUT/2122/cla2122.csv")
tab2122[2] = pd.read_csv("dataBRUT/2122/res2122.csv")
tab2122[3] = pd.read_csv("dataBRUT/2122/evo2122.csv")
tab2122[4] = pd.read_csv("dataBRUT/2122/for2122.csv")

tab2223 = ['', '', '', '', '']
tab2223[0] = pd.read_csv("dataBRUT/2223/pre2223.csv")
tab2223[1] = pd.read_csv("dataBRUT/2223/cla2223.csv")
tab2223[2] = pd.read_csv("dataBRUT/2223/res2223.csv")
tab2223[3] = pd.read_csv("dataBRUT/2223/evo2223.csv")
tab2223[4] = pd.read_csv("dataBRUT/2223/for2223.csv")

tab2324 = ['', '', '', '', '']
tab2324[0] = pd.read_csv("dataBRUT/2324/pre2324.csv")
tab2324[1] = pd.read_csv("dataBRUT/2324/cla2324.csv")
tab2324[2] = pd.read_csv("dataBRUT/2324/res2324.csv")
tab2324[3] = pd.read_csv("dataBRUT/2324/evo2324.csv")
tab2324[4] = pd.read_csv("dataBRUT/2324/for2324.csv")

tab2425 = ['', '', '', '', '']
tab2425[0] = pd.read_csv("dataBRUT/2425/pre2425.csv")
tab2425[1] = pd.read_csv("dataBRUT/2425/cla2425.csv")
tab2425[2] = pd.read_csv("dataBRUT/2425/res2425.csv")
tab2425[3] = pd.read_csv("dataBRUT/2425/evo2425.csv")
tab2425[4] = pd.read_csv("dataBRUT/2425/for2425.csv")

<a id="nettoyage"></a>
### Nettoyage

La fonction nettoyage de cleanData.py uniformise les données du scraping. Les noms des équipes sont nettoyés, on se débarrasse des notes, et les classements sont tous mis sous forme numérique (ex: 1er -> 1, 2ème -> 2...). Les valeurs manquantes sont aussi remplacées par un tiret.

In [ ]:
from cleanData import nettoyage
nettoyage(None, tab0506[0], tab0506[1], None, None, do_pre=False, do_evo=False, do_for=False)
nettoyage(None, tab0607[0], tab0607[1], None, None, do_pre=False, do_evo=False, do_for=False)
nettoyage(None, tab0708[0], tab0708[1], None, None, do_pre=False, do_evo=False, do_for=False)
nettoyage(None, tab0809[0], tab0809[1], None, None, do_pre=False, do_evo=False, do_for=False)
nettoyage(tab0910[0], tab0910[1], tab0910[2],None, None, do_evo=False, do_for=False)
nettoyage(tab1011[0], tab1011[1], tab1011[2],tab1011[3], None, do_for=False)
nettoyage(tab1112[0], tab1112[1], tab1112[2],tab1112[3], None, do_for=False)
nettoyage(tab1213[0], tab1213[1], tab1213[2],tab1213[3], None, do_for=False)
nettoyage(tab1314[0], tab1314[1], tab1314[2],tab1314[3], None, do_for=False)
nettoyage(tab1415[0], tab1415[1], tab1415[2],tab1415[3], None, do_for=False)
nettoyage(tab1516[0], tab1516[1], tab1516[2],tab1516[3], None, do_for=False)
nettoyage(*tab1617)
nettoyage(*tab1718)
nettoyage(*tab1819)
nettoyage(*tab1920)
nettoyage(*tab2021)
nettoyage(*tab2122)
nettoyage(*tab2223)
nettoyage(*tab2324)
nettoyage(*tab2425)

<a id="uniformisation"></a>

### Uniformisation

On veut pour chaque année les tableaux : présentation, classement, résultats, évolution, forme


In [ ]:
tab0506 = [None, tab0506[0], tab0506[1], None, None]
tab0607 = [None, tab0607[0], tab0607[1], None, None]
tab0708 = [None, tab0708[0], tab0708[1], None, None]
tab0809 = [None, tab0809[0], tab0809[1], None, None]
tab0910 = [tab0910[0], tab0910[1], tab0910[2], None, None]
tab1011 = [tab1011[0], tab1011[1], tab1011[2], tab1011[3], None]
tab1112 = [tab1112[0], tab1112[1], tab1112[2], tab1112[3], None]
tab1213 = [tab1213[0], tab1213[1], tab1213[2], tab1213[3], None]
tab1314 = [tab1314[0], tab1314[1], tab1314[2], tab1314[3], None]
tab1415 = [tab1415[0], tab1415[1], tab1415[2], tab1415[3], None]
tab1516 = [tab1516[0], tab1516[1], tab1516[2], tab1516[3], None]

In [ ]:
# On va maintenant fusionner les tableaux, on change le nom pour faciliter la boucle :
tab1 = tab0506
tab2 = tab0607
tab3 = tab0708
tab4 = tab0809
tab5 = tab0910
tab6 = tab1011
tab7 = tab1112
tab8 = tab1213
tab9 = tab1314
tab10 = tab1415
tab11 = tab1516
tab12 = tab1617
tab13 = tab1718
tab14 = tab1819
tab15 = tab1920
tab16 = tab2021
tab17 = tab2122
tab18 = tab2223
tab19 = tab2324
tab20 = tab2425

In [ ]:
colonnes = ["Club","Dernière montée","Budget en M€","Classement précédent","Entraîneur en chef","Stade","Capacité"]
def init_df():
    return pd.DataFrame([[np.nan]*len(colonnes)], columns=colonnes)

for tab in (tab1, tab2, tab3, tab4):
    if tab[0] is None:
        tab[0] = init_df()

<a id="uniformisation"></a>

### Travail sur calendrier

Un problème se pose pour les tableaux évolution et forme qui n'apparaissent plus respectivement à partir de 2011 et 2016, il faut donc les reconstruire à partir de résultats mais ce dernier ne donne que les scores des matchs sans l'ordre des journées, il faut donc rajouter ces informations pour avoir des informations pour chaque journées (classement à la fin de la journée pour évolution, victoire/nul/défaite pour forme)

In [ ]:
from calendrierData import make_calBIN_from_mapped, build_points_diff_table, build_rank_table, reshape_table

Un gros travail de mapping doit être fait pour uniformiser le nom d'équipe comme : USAP, USA Perpignan, Perpignan ; ou même encore plus simple les cas : Stade Toulousain et Stade toulousain qui ne sont pas reconnus pareil ce qui va poser des problèmes pour mener des études statistiques dessus

In [ ]:
mapping = {
    "Montferrand": "ASM Clermont",
    'Paris' : 'Stade français Paris',
    'Stade français': 'Stade français Paris',
    'Clermont': 'ASM Clermont',
    'La Rochelle': 'Stade rochelais',
    'Toulouse': 'Stade toulousain',
    'Stade toulousain T': 'Stade toulousain',
    'Bayonne': 'Aviron bayonnais',
    'Brive': 'CA Brive',
    'Montpellier': 'Montpellier HR',
    'Montpellier RC': 'Montpellier HR',
    'Toulon': 'RC Toulon',
    'Castres': 'Castres olympique',
    'Pau': 'Section paloise',
    'Agen': 'SU Agen',
    'Grenoble': 'FC Grenoble',
    'Oyonnax': 'US Oyonnax',
    'Perpignan': 'USA Perpignan',
    'Bordeaux-Bègles': 'Union Bordeaux Bègles',
    'Bordeaux Bègles' : 'Union Bordeaux Bègles',
    'Lyon' : 'Lyon OU',
    'ASM Clermont Auvergne': 'ASM Clermont',
    'Biarritz O.': 'Biarritz Olympique',
    'Biarritz olympique': 'Biarritz Olympique',
    'Bourgoin-Jallieu': 'CS Bourgoin-Jallieu',
    'Castres O.': 'Castres olympique',
    'Montpellier RC': 'Montpellier HR',
    'Oyonnax Rugby': 'US Oyonnax',
    'Racing Métro 92': 'Racing 92',
    'Montauban TGXV': 'US Montauban',
    'Stade montois': 'Stade Montois',
    "Auch": "FC Auch Gers",
    "FC Auch": "FC Auch Gers",
    'Albi': 'SC Albi',
    'Biarritz': 'Biarritz Olympique',
    'Bourgoin': 'CS Bourgoin-Jallieu',
    'Dax': 'US Dax',
    'Mont de Marsan': 'Stade Montois',
    'Montauban': 'US Montauban',
    'Narbonne': 'RC Narbonne',
    'Stade Français': 'Stade français Paris',
    'Stade Toulousain': 'Stade toulousain',
    'USAP': 'USA Perpignan',
    "Racing Metro 92": "Racing 92",
    "LOU" : "Lyon OU"}

abbr_mapping = {
    "AGE": "SU Agen",
    "BAY": "Aviron bayonnais",
    "BIA": "Biarritz Olympique",
    "BOU": "CS Bourgoin-Jallieu",
    "BRI": "CA Brive",
    "CAS": "Castres olympique",
    "CLE": "ASM Clermont",
    "MPL": "Montpellier HR",
    "NAR": "RC Narbonne",
    "PAU": "Section paloise",
    "PER": "USA Perpignan",
    "STF": "Stade français Paris",
    "TLN": "RC Toulon",
    "TLS": "Stade toulousain",

    "ALB": "SC Albi",
    "AUC": "FC Auch Gers",
    "DAX": "US Dax",
    "MDM": "Stade Montois",
    "MTB": "US Montauban",  

    "UBB": "Union Bordeaux Bègles",
    "CAB": "CA Brive",
    "MHR": "Montpellier HR",        # alias qui pointe vers le libellé exact de res
    "SFR": "Stade français Paris",
    "TOU": "Stade toulousain",
    "GRE": "FC Grenoble",
    "LOU": "Lyon OU",
    "RAC": "Racing 92",
    "ROC": "Stade rochelais",
    "OYO": "US Oyonnax",
    "VAN": "RC Vannes",

    "BOB": "Union Bordeaux Bègles",   
    "MRC": "Montpellier HR",
    "RCT": "RC Toulon",
    "SCA": "SC Albi",
    "SUA": "SU Agen",
    "USM": "US Montauban",
    "USO": "US Oyonnax",
    
    "BOR": "Union Bordeaux Bègles",
    "LAR": "Stade rochelais",
    "LYO": "Lyon OU",
    "MON": "Montpellier HR",
}

cal_lower_mapping = {
    "agen": "su agen",
    "albi": "sc albi",
    "auch": "fc auch gers",
    "bayonne": "aviron bayonnais",
    "biarritz": "biarritz olympique",
    "bourgoin": "cs bourgoin-jallieu",
    "brive": "ca brive",
    "castres": "castres olympique",
    "clermont": "asm clermont",
    "dax": "us dax",
    "mont de marsan": "stade montois",
    "montauban": "us montauban",
    "montferrand": "asm clermont",
    "montpellier": "montpellier hr",
    "narbonne": "rc narbonne",
    "pau": "section paloise",
    "stade français": "stade français paris",
    "stade toulousain": "stade toulousain",
    "toulon": "rc toulon",
    "usap": "usa perpignan",
    "la rochelle": "stade rochelais",
    "racing metro 92":"racing 92",
    "racing métro 92":"racing 92",
    "lou": "lyon ou",
    "grenoble": "fc grenoble",
    "oyonnax": "us oyonnax",
}

cal_upper_mapping = {
    "AGEN": "SU AGEN",
    "ALBI": "SC ALBI",
    "AUCH": "FC AUCH GERS",
    "BAYONNE": "AVIRON BAYONNAIS",
    "BIARRITZ": "BIARRITZ OLYMPIQUE",
    "BOURGOIN": "CS BOURGOIN-JALLIEU",
    "BRIVE": "CA BRIVE",
    "CASTRES": "CASTRES OLYMPIQUE",
    "CLERMONT": "ASM CLERMONT",
    "DAX": "US DAX",
    "MONT DE MARSAN": "STADE MONTOIS",
    "MONTAUBAN": "US MONTAUBAN",
    "MONTFERRAND" : "ASM CLERMONT",
    "MONTPELLIER": "MONTPELLIER HR",
    "NARBONNE": "RC NARBONNE",
    "PAU": "SECTION PALOISE",
    "LA ROCHELLE": "STADE ROCHELAIS",
    "STADE FRANÇAIS": "STADE FRANÇAIS PARIS",
    "STADE TOULOUSAIN": "STADE TOULOUSAIN",
    "TOULON": "RC TOULON",
    "USAP": "USA PERPIGNAN",
    "RACING METRO 92":"RACING 92",
    "RACING MÉTRO 92":"RACING 92",
    "LOU": "LYON OU",
    "GRENOBLE": "FC GRENOBLE",
    "OYONNAX": "US OYONNAX",
}

In [ ]:
normalized_res = {}

for i in range(1, 12):  # de tab1 à tab11 inclus
    df = globals()[f"tab{i}"][2].copy()
    
    # Remplacer abréviations (ligne 0, colonnes sauf la première)
    df.iloc[0, 1:] = df.iloc[0, 1:].replace(abbr_mapping)
    
    # Remplacer noms domicile (première colonne sauf l’entête)
    df.iloc[1:, 0] = df.iloc[1:, 0].replace(mapping)
    
    # Sauvegarder
    normalized_res[f"res{i:02d}"] = df

In [ ]:
# saisons à traiter
seasons = ["0506","0607","0708","0809","0910","1011","1112","1213","1314","1415","1516"]

def map_team_name(cell: str):
    if pd.isna(cell) or cell == "":
        return cell
    s = str(cell).strip()
    if s.isupper():
        # MAJUSCULE = la colonne joue à domicile
        return cal_upper_mapping.get(s, s)
    else:
        # minuscules = la colonne joue à l'extérieur
        return cal_lower_mapping.get(s, s)

cal_norm = {}

for season in seasons:
    df = pd.read_csv(f"dataLLM/{season}cal.csv")

    # 1) appliquer le mapping cellule par cellule sur les colonnes équipes
    team_cols = df.columns[1:]
    for col in team_cols:
        df[col] = df[col].apply(map_team_name)

    # 2) renommer la première colonne en "Journées"
    df.rename(columns={df.columns[0]: "Journées"}, inplace=True)

    # 3) renommer les en-têtes d'équipes avec col_to_resname
    #    (on garde tel quel si la clé n'est pas dans le mapping)
    header_map = {col: mapping.get(col, col) for col in team_cols}
    df.rename(columns=header_map, inplace=True)

    cal_norm[season] = df

In [ ]:
calBINs = {}

for i, year in enumerate(range(506, 1517, 101)):  # 0506, 0607, ..., 1516
    year_str = str(year).zfill(4)      # "0506", "0607", ...
    res_key = f"res{str(i+1).zfill(2)}"  # "res01", "res02", ...
    
    calBINs[year_str] = make_calBIN_from_mapped(
        cal_norm[year_str],
        normalized_res[res_key]
    )

Avec calBIN on a complètement les matrices forme, il manque à les uniformiser dans le format des autres, i.e journées en colonnes et aussi à calculer les matrices evolution

In [ ]:
ranks = {}

for year_str, calBIN in calBINs.items():
    # trouver le bon res_key à partir de l’année
    # ex: "0506" → "res01", "0607" → "res02", etc.
    idx = list(calBINs.keys()).index(year_str) + 1
    res_key = f"res{str(idx).zfill(2)}"
    
    points_diff = build_points_diff_table(calBIN, normalized_res[res_key])
    ranks[year_str] = build_rank_table(points_diff)

In [ ]:
evos = {}
fors = {}

for year in ranks.keys():  # "0506", "0607", ..., "1516"
    # Pour le classement (rank → evo)
    evos[f"evo{year}"] = reshape_table(ranks[year])

    # Pour les résultats (calBIN → for)
    fors[f"for{year}"] = reshape_table(calBINs[year])

In [ ]:
for i in range(506, 911, 101):  # 0506 -> 0910, par pas de 101
    key = f"evo{str(i).zfill(4)}"  # génère "evo0506", etc.
    tab_name = f"tab{str(i).zfill(4)}"  # génère "tab0506", etc.
    globals()[tab_name][3] = evos[key]

In [ ]:
for i in range(506, 1517, 101):  # 0506 -> 1516, par pas de 101
    key = f"for{str(i).zfill(4)}"       # ex: "for0506"
    tab_name = f"tab{str(i).zfill(4)}"  # ex: "tab0506"
    globals()[tab_name][4] = fors[key]

<a id="regroupement"></a>
### Regroupement

On fusionne tous les tableaux pour travailler plus facilement avec par la suite

In [ ]:
# On crée une fonction qui permet d'ajouter une colonne année à chaque tableau pour bien pouvoir les séparer par année si besoin après
def ajout_an(df,i):
   df["année"] = 2005 + i
   return df

# On va faire maintenant une boucle pour concaténer les tableaux entre eux :
tableauglobal = [pd.DataFrame() for _ in range(5)]

for j in range(1, 21):
    for i in range(5):
         tableauglobal[i] = pd.concat([tableauglobal[i], ajout_an(eval(f"tab{j}")[i], j - 1)], ignore_index=True)

tab_presentation_global = tableauglobal[0]
tab_classement_global = tableauglobal[1]
tab_resultat_global = tableauglobal[2]
tab_evolution_classement_global = tableauglobal[3]
tab_forme_global = tableauglobal[4]
# On convertit aussi ceraines données au format numérique
tab_presentation_global['Budget en M€'] = pd.to_numeric(tab_presentation_global['Budget en M€'], errors='coerce')
tab_presentation_global['Classement précédent'] = pd.to_numeric(tab_presentation_global['Classement précédent'], errors='coerce')
#on enleve les journées de rattrapage inutiles
tab_forme_global = tab_forme_global.iloc[:, :-3]

On vérifie que le mapping a bien été fait

In [ ]:
presentation = tab_presentation_global
classement = tab_classement_global
evolution = tab_evolution_classement_global
forme = tab_forme_global
resultat = tab_resultat_global
resultat.rename(columns={resultat.columns[0]: "Club"}, inplace=True)
resultat = resultat.fillna("-")

# On commence déjà par uniformiser le nom de la colonne 'Club':
evolution.rename(columns={'Equipes/Journées': 'Club'}, inplace=True)
forme.rename(columns={'Equipes/Journées': 'Club'}, inplace=True)

# On a plus qu'à uniformiser les noms dans tout les tableaux
presentation['Club'] = presentation['Club'].replace(mapping)
classement['Club'] = classement['Club'].replace(mapping)
forme['Club'] = forme['Club'].replace(mapping)
evolution['Club'] = evolution['Club'].replace(mapping)

#pour resultat c'est plus compliqué car toutes les 15 lignes on a un mapping différent du précédent
resultat = resultat.rename(columns={0: "Club"})

# Parcours des lignes 0, 15, 30, ...
for i in range(0, len(resultat), 15):
    if i < len(resultat):
        resultat.iloc[i, 1:-1] = resultat.iloc[i, 1:-1].replace(abbr_mapping)

resultat['Club'] = resultat['Club'].apply(
    lambda x: mapping[x] if x in mapping and x != "Clubs" else x
)

On va ensuite sauvegarder tous les tableaux dans le dossier data

In [ ]:
# presentation.to_csv('data/presentation.csv',index=False)
# classement.to_csv('data/classement.csv',index=False)
# forme.to_csv('data/forme.csv',index=False)
# evolution.to_csv('data/evolution.csv',index=False)
# resultat.to_csv('data/resultat.csv',index=False)

Ou les récupérer

In [ ]:
presentation=pd.read_csv('data/presentation.csv')
classement=pd.read_csv('data/classement.csv')
forme=pd.read_csv('data/forme.csv')
evolution=pd.read_csv('data/evolution.csv')
resultat=pd.read_csv('data/resultat.csv')

<a id="stats"></a>
# Statistiques Descriptives

<a id="visual"></a>
### Visualisation

La fonction club de computeData permet d'obtenir les statistiques descriptives pour un club en particulier, pour chaque année qu'il passe dans le top 14: son budget en M€, son entraîneur, son rang, et le nombre de recontres jouées, dont les victoires et les défaites.


In [ ]:
from computeData import club

toulouse=club("Stade toulousain", presentation, classement)
toulouse

In [ ]:
perpignan=club("USA Perpignan", presentation, classement)
perpignan

On peut visualiser l'évolution du budget des clubs au sein du championnat

In [ ]:
plt.figure(figsize=(12, 6))
df_plot = presentation[presentation["Budget en M€"].notna()]
sns.boxplot(x="année", y="Budget en M€", data=df_plot, palette="YlOrBr")

plt.title("Distribution du Budget des Clubs de 2009 à 2024")
plt.xlabel("Année")
plt.ylabel("Budget en M€")

plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
budget2024 = presentation[presentation["année"] == 2024]
budget2024[["Club", "Budget en M€"]].sort_values(by="Budget en M€",ascending=False)

On visualise l'évolution du trio de tête :

In [ ]:
import matplotlib.ticker as mticker

clubs_selectionnes = ["Stade toulousain","Stade rochelais","RC Toulon","Union Bordeaux Bègles"]

couleurs = {"Stade toulousain": "red","Stade rochelais": "gold","RC Toulon": "black","Union Bordeaux Bègles": "blue"}


# filtrer pour le top et uniquement ces clubs
top_clubs = (
    classement.sort_values(by=["année", "Rang"])
    .groupby("année")
    .head(14)
    .query("Club in @clubs_selectionnes")
)

plt.figure(figsize=(14, 8))
for club in top_clubs["Club"].unique():
    club_data = top_clubs[top_clubs["Club"] == club]
    plt.plot(club_data["année"], club_data["Rang"], marker="o", label=club, color=couleurs[club], linewidth=2)
    
plt.gca().invert_yaxis()  # mettre 1 en haut
plt.ylim(14.3, 0.7)

plt.title("Évolution du classement des clubs sélectionnés (2005-2024)")
plt.xlabel("Saison")
plt.gca().xaxis.set_major_locator(mticker.MaxNLocator(integer=True))
plt.ylabel("Classement")
plt.legend(title="Club", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.grid(True)
plt.tight_layout()
plt.show()


<a id="Suivi des performances"></a>
### Suivi des performances

Continuons par analyser d'autres graphiques, en étudiant en particulier les tables évolution et forme :


In [ ]:
from computeData import plot_club_evolution

plot_club_evolution("Stade toulousain", evolution, forme)

In [ ]:
plot_club_evolution("Union Bordeaux Bègles", evolution, forme)

In [ ]:
plot_club_evolution("USA Perpignan", evolution, forme)

<a id="predict"></a>
# Modèle prédictif

Après un aperçu des données à notre disposition, nous allons chercher à répondre à 2 questions : 
- peut-on prédire le champion de la saison, i.e à la fin des 26 journées, savoir qui remporte les phases finales ?
- peut-on prédire une saison entière avant qu'elle ne commence, et comment évolue cette prédiction au fur et à mesure que les journées soient connues ?

<a id="predictionChampion"></a>
### Prédiction du champion

Tout d'abord affichons le champion des dernières années :


In [ ]:
#packages:
from io import StringIO
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import sys
import subprocess
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

#lxml
subprocess.check_call([sys.executable, "-m", "pip", "install", "lxml"])

# pour partie predict
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

presentation=pd.read_csv('data/presentation.csv')
classement=pd.read_csv('data/classement.csv')
forme=pd.read_csv('data/forme.csv')
evolution=pd.read_csv('data/evolution.csv')
resultat=pd.read_csv('data/resultat.csv')

In [ ]:
import pandas as pd

# Données champions avec noms normalisés (conformes à ton mapping)
data_enligne = [
    {"année": 2005, "champion": "Biarritz Olympique"},
    {"année": 2006, "champion": "Stade français Paris"},
    {"année": 2007, "champion": "Stade toulousain"},
    {"année": 2008, "champion": "USA Perpignan"},
    {"année": 2009, "champion": "ASM Clermont"},
    {"année": 2010, "champion": "Stade toulousain"},
    {"année": 2011, "champion": "Stade toulousain"},
    {"année": 2012, "champion": "Castres olympique"},
    {"année": 2013, "champion": "RC Toulon"},
    {"année": 2014, "champion": "Stade français Paris"},
    {"année": 2015, "champion": "Racing 92"},
    {"année": 2016, "champion": "ASM Clermont"},
    {"année": 2017, "champion": "Castres olympique"},
    {"année": 2018, "champion": "Stade toulousain"},
    {"année": 2019, "champion": None},  # saison annulée
    {"année": 2020, "champion": "Stade toulousain"},
    {"année": 2021, "champion": "Montpellier HR"},
    {"année": 2022, "champion": "Stade toulousain"},
    {"année": 2023, "champion": "Stade toulousain"},
    {"année": 2024, "champion": "Stade toulousain"},
]

# Vice-champions avec noms normalisés
vice_champions = {
    2005: "Stade français Paris",
    2006: "ASM Clermont",
    2007: "ASM Clermont",
    2008: "ASM Clermont",
    2009: "USA Perpignan",
    2010: "USA Perpignan",
    2011: "Montpellier HR",
    2012: "Stade toulousain",
    2013: "Castres olympique",
    2014: "Castres olympique",
    2015: "ASM Clermont",
    2016: "RC Toulon",
    2017: "RC Toulon",
    2018: "ASM Clermont",
    2019: None,  # annulée
    2020: "Stade rochelais",
    2021: "Castres olympique",
    2022: "Stade rochelais",
    2023: "Stade rochelais",
    2024: "Union Bordeaux Bègles",
}

# Compléter le dataframe
for row in data_enligne:
    row["vice_champion"] = vice_champions.get(row["année"], None)

df_champions = pd.DataFrame(data_enligne)
df_champions

# Prédiction du champion

In [ ]:
from prediction_cleandata import extract_features_top14

In [ ]:
features = extract_features_top14(classement, evolution, forme, df_champions, resultats=resultat)

In [ ]:
features

<a id="regressionlogistique"></a>

### Regression Logistique

In [ ]:
from prediction_reg import fit_predict_logit, evaluate_results

In [ ]:
results, probs = fit_predict_logit(features, df_champions)

In [ ]:
results

In [ ]:
summary = evaluate_results(results, probs)
summary

<a id="randomForest"></a>

### Random Forest

In [ ]:
from prediction_reg import run_random_forest_simple

In [ ]:
pred_df_RF, year_table_RF, metrics_RF = run_random_forest_simple(
    features, df_champions,
    calibrate=False,         # commence SANS calibration
    blend_prior=True,        # active le prior de rang
    prior_kind="rank",       # "rank" ou "points"
    prior_temp=0.8,          # 0.6 ↔ 1.2 à tester
    alpha=0.30               # poids du prior (0.2–0.5 à tester)
)

In [ ]:
year_table_RF

In [ ]:
metrics_RF

In [ ]:
# --- 2) Mini-grid simple à tester ---
configs = [
    dict(name="base_like",   n_estimators=1200, max_depth=10, max_features="sqrt", min_samples_leaf=2, class_weight="balanced", criterion="gini"),
    dict(name="more_trees",  n_estimators=2000, max_depth=10, max_features="sqrt", min_samples_leaf=2, class_weight="balanced", criterion="gini"),
    dict(name="deeper",      n_estimators=1200, max_depth=None, max_features="sqrt", min_samples_leaf=2, class_weight="balanced", criterion="gini"),
    dict(name="log2_feats",  n_estimators=1200, max_depth=10, max_features="log2", min_samples_leaf=2, class_weight="balanced", criterion="gini"),
    dict(name="feat_60pc",   n_estimators=1200, max_depth=10, max_features=0.6,   min_samples_leaf=2, class_weight="balanced", criterion="gini"),
    dict(name="leaf1",       n_estimators=1200, max_depth=10, max_features="sqrt", min_samples_leaf=1, class_weight="balanced", criterion="gini"),
    dict(name="no_weight",   n_estimators=1200, max_depth=10, max_features="sqrt", min_samples_leaf=2, class_weight=None,      criterion="gini"),
    dict(name="entropy",     n_estimators=1200, max_depth=10, max_features="sqrt", min_samples_leaf=2, class_weight="balanced", criterion="entropy"),
    # variantes prior
    dict(name="alpha_0.2",   n_estimators=1200, max_depth=10, max_features="sqrt", min_samples_leaf=2, class_weight="balanced", criterion="gini"),
    dict(name="alpha_0.4",   n_estimators=1200, max_depth=10, max_features="sqrt", min_samples_leaf=2, class_weight="balanced", criterion="gini"),
]
# prior_kind="rank" et prior_temp=0.8 fixés ; on bougera alpha juste après

rows = []
for cfg in configs:
    alpha = 0.30
    if cfg["name"] == "alpha_0.2": alpha = 0.20
    if cfg["name"] == "alpha_0.4": alpha = 0.40

    _, year_table, metrics = run_random_forest_simple(
        features, df_champions,
        calibrate=False,
        blend_prior=True,
        prior_kind="rank",
        prior_temp=0.8,
        alpha=alpha,
        n_estimators=cfg["n_estimators"],
        max_depth=cfg["max_depth"],
        max_features=cfg["max_features"],
        min_samples_leaf=cfg["min_samples_leaf"],
        class_weight=cfg["class_weight"],
        criterion=cfg["criterion"],
        random_state=42,
    )
    rows.append({
        "config": cfg["name"],
        "top1": metrics["top1"],
        "top2": metrics["top2"],
        "brier": metrics["brier"],
        "logloss": metrics["logloss"],
        "params": {k:v for k,v in cfg.items() if k!="name"},
        "alpha": alpha
    })

rf_summary = pd.DataFrame(rows).sort_values(["top1","top2"], ascending=[False, False]).reset_index(drop=True)
rf_summary

<a id="xgboost"></a>

### XGBoost

In [ ]:
from prediction_reg import run_xgb_loso_simple

In [ ]:
year_table_XG, metrics_XG = run_xgb_loso_simple(features, df_champions, target="is_champion")

In [ ]:
metrics_XG

In [ ]:
year_table_XG

In [ ]:
configs = [
    # base (ta config)
    dict(name="base", n_estimators=400, max_depth=4, learning_rate=0.05, subsample=0.9, colsample_bytree=0.9, reg_lambda=1.0),

    # un peu plus profond
    dict(name="depth5", n_estimators=400, max_depth=5, learning_rate=0.05, subsample=0.9, colsample_bytree=0.9, reg_lambda=1.0),
    dict(name="depth6", n_estimators=400, max_depth=6, learning_rate=0.05, subsample=0.9, colsample_bytree=0.9, reg_lambda=1.0),

    # plus d'arbres + LR plus faible
    dict(name="lr0.04_ne600", n_estimators=600, max_depth=4, learning_rate=0.04, subsample=0.9, colsample_bytree=0.9, reg_lambda=1.0),
    dict(name="lr0.03_ne800", n_estimators=800, max_depth=4, learning_rate=0.03, subsample=0.9, colsample_bytree=0.9, reg_lambda=1.0),

    # échantillonnage un poil différent
    dict(name="sub08_col08", n_estimators=400, max_depth=4, learning_rate=0.05, subsample=0.8, colsample_bytree=0.8, reg_lambda=1.0),

    # régularisation L2 un peu plus forte
    dict(name="lambda2", n_estimators=400, max_depth=4, learning_rate=0.05, subsample=0.9, colsample_bytree=0.9, reg_lambda=2.0),
]

rows = []
all_year_tables = {}

for cfg in configs:
    yt, m = run_xgb_loso_simple(
        features, df_champions, target="is_champion",
        n_estimators=cfg["n_estimators"],
        max_depth=cfg["max_depth"],
        learning_rate=cfg["learning_rate"],
        subsample=cfg["subsample"],
        colsample_bytree=cfg["colsample_bytree"],
        reg_lambda=cfg["reg_lambda"],
        random_state=42,
    )
    rows.append({
        "config": cfg["name"],
        "precision_top1": float(m.loc[0,"precision_top1"]),
        "precision_top2": float(m.loc[0,"precision_top2"]),
        "params": {k:v for k,v in cfg.items() if k!="name"}
    })
    all_year_tables[cfg["name"]] = yt

summary = pd.DataFrame(rows).sort_values(["precision_top1","precision_top2"], ascending=[False, False]).reset_index(drop=True)
print(summary)
# Pour voir le détail d'une config donnée (ex. la meilleure) :
best_name = summary.loc[0, "config"]
print(f"\n--- Détails meilleure config: {best_name} ---")
all_year_tables[best_name]


Vérification

In [ ]:
def verif_premier(year_table: pd.DataFrame, classement: pd.DataFrame) -> pd.DataFrame:
    """
    Compare la prédiction du modèle avec l'équipe classée 1ʳᵉ au classement régulier.

    Args:
        year_table (pd.DataFrame): Résultats par saison (année, predicted, actual, etc.)
        classement (pd.DataFrame): Classement complet avec colonnes 'année', 'Club', 'Rang'

    Returns:
        pd.DataFrame: Tableau enrichi avec la colonne 'premier_saison'
                      (club classé 1er) et 'same_as_first' (booléen)
    """
    # S'assurer que la colonne s'appelle 'année'
    if "annee" in year_table.columns:
        year_table = year_table.rename(columns={"annee": "année"})
    if "annee" in classement.columns:
        classement = classement.rename(columns={"annee": "année"})

    # Extraire l'équipe en Rang=1 pour chaque saison
    premiers = classement.loc[classement["Rang"] == 1, ["année", "Club"]].rename(columns={"Club": "premier_saison"})

    # Merge avec le tableau des prédictions
    merged = year_table.merge(premiers, on="année", how="left")

    # Vérifier si la prédiction correspond au premier
    merged["same_as_first"] = (merged["predicted"] == merged["premier_saison"]).astype(int)

    return merged[["année", "predicted", "actual", "premier_saison", "same_as_first"]]

In [ ]:
verif_premier(year_table_RF, classement)

In [ ]:
verif_premier(year_table_XG, classement)

<a id="predictionElo"></a>


# Prédiction par Elo

In [ ]:
#packages:
from io import StringIO
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import sys
import subprocess
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

presentation=pd.read_csv('data/presentation.csv')
classement=pd.read_csv('data/classement.csv')
forme=pd.read_csv('data/forme.csv')
evolution=pd.read_csv('data/evolution.csv')
resultat=pd.read_csv('data/resultat.csv')

In [ ]:
from prediction_elo import find_best_parameters

find_best_parameters(classement, resultat)

<a id="test2425"></a>

### TEST sur la saison 24/25

In [ ]:
from prediction_elo import elo_preseason_with_mercato

# 2) Tes meilleurs paramètres (SEQUENTIAL updates — deterministic order)
H_BEST = 63.837
S_BEST = 259.415
NU_BEST = 0.031
K_BEST = 27.936
MARGIN_SCALE_BEST = 2.485

W_LAST  = 0.68   # => w_recent
W_PREV  = 1 - W_LAST  # = 0.32
PROMO_PENALTY_BEST = 0.057
BOTTOM_K_BEST = 3

# (Optionnel) notes de mercato si tu en as :
notes_mercato_2425 = {
    "Stade toulousain":        6.2,  # stabilité + Naoto Saito, Efrain Elías
    "Union Bordeaux Bègles":   8.0,  # Carbery, Jonny Gray, Swinton, Retière
    "Stade français Paris":    5.3,  # Carbonel, Foursans, Tanga, Nicotera
    "Stade rochelais":         6.8,  # Kane Douglas, Vunivalu (ailier)
    "RC Toulon":               8.2,  # Sinckler, Ludlam, Frisch, Lucchesi
    "Castres olympique":       6.8,  # Collier, Ducat, Jedrasiak, Matkava
    "Racing 92":               8.9,  # Owen Farrell, Bamba, Taofifénua, Dayimani
    "ASM Clermont":            7.0,  # Ala'alatoa, Tauzin, Hamdaoui
    "Section paloise":         7.2,  # Klemenczak, Aymeric Luc, Picquette, Kaulashvili
    "USA Perpignan":           7.1,  # Brookes, Devaux, Aprasidze, Buliruarua, Warion
    "Lyon OU":                 6.9,  # Ainsley, Lavanini, Matavesi, Gomes Sa (+ Chat)
    "Aviron bayonnais":        9.5,  # Tuilagi, Segonds, Germain, Alex Moon
    "Montpellier HR":          6.5,  # Vunipola, Hogg, Uelese, Miotti, Abuladze
    "RC Vannes":               7.3,  # Mako Vunipola, Nakosi, Tani Vili, Varney, Medrano, Rayasi
}

# 3) Appel de la fonction
elo_with_notes, elo_preseason_df, classement_corrige_2324, table_attendue_2425 = elo_preseason_with_mercato(
    classement=classement,
    forme=forme,
    resultat=resultat,                 # nécessaire pour calculer la table attendue
    base_year=2023,                      # 23/24 vient de finir → on prédit 24/25
    # --- hyperparams pré-saison (tu peux laisser les défauts) ---
    w_last=W_LAST,                       # 75% saison 23/24
    w_prev=W_PREV,                       # 25% saison 22/23
    rho=0.80,                          # régression vers la moyenne
    bottom_k=BOTTOM_K_BEST,
    promu="RC Vannes",
    relegue="US Oyonnax",
    m_equipes=14,
    promo_penalty=PROMO_PENALTY_BEST,             # issu de ta recherche
    # delta_niveau=-35.0,              # ajuste si tu as estimé un écart de division
    # notes_mercato=notes_mercato,     # décommente si tu fournis un dict
    # --- shrink par étages (laisse par défaut si tu veux) ---
    apply_tier_compress=True,
    # --- proba match (Davidson) pour la table attendue ---
    H=H_BEST,
    s=S_BEST,
    nu=NU_BEST,
    compute_expected_table=True,
    season_for_expected=2024,          # saison 24/25 codée 2024 dans resultat.csv
    do_print=False,
    #mercato
    gamma=6,
    notes_mercato=notes_mercato_2425
)

# (Optionnel) Aperçu rapide
print("\nTOP pré-saison (après mercato & shrink):")
print(elo_with_notes.head(5))
print("\nClassement attendu 24/25 (points attendus):")
if table_attendue_2425 is not None:
    print(table_attendue_2425)

In [ ]:
elo_with_notes

<a id="saison2526"></a>

### Prédiction sur 25/26

In [ ]:
notes_mercato_2526 = {
    "USA Perpignan": 9,
    "Stade rochelais": 8,
    "Union Bordeaux Bègles": 8,
    "Lyon OU": 7.5,
    "Stade toulousain": 6.5,
    "Aviron bayonnais": 6.0,
    "RC Toulon": 5.5,
    "Section paloise": 5.5,
    "Montpellier HR": 5.5,
    "Castres olympique": 5.0,
    "Racing 92": 5.0,
    "US Montauban": 4.0,
    "Stade français Paris": 4.5,
    "ASM Clermont": 4.5,
}

elo_2526, elo_pre_2526, cls_corr_2425, table_2526 = elo_preseason_with_mercato(
    classement=classement,
    forme=forme,
    resultat=resultat,
    base_year=2024,                      # 24/25 vient de finir → on prédit 25/26
    promu="US Montauban",
    relegue="RC Vannes",
    w_last=W_LAST, w_prev=W_PREV, rho=0.80,
    H=H_BEST, s=S_BEST, nu=NU_BEST, bottom_k=BOTTOM_K_BEST,
    notes_mercato=notes_mercato_2526,                  
    do_print=False
)

In [ ]:
elo_2526

In [ ]:
table_2526

In [ ]:
calendrier2526 = pd.read_csv("data2526/calendrier_2526.csv")
calendrier2526

In [ ]:
from prediction_elo import print_probas_journee

print_probas_journee(elo_2526, calendrier2526, journee=1, H=H_BEST, s=S_BEST, nu=NU_BEST)

In [ ]:
from prediction_elo import add_scores_for_journee

In [ ]:
# 7 matchs pour une journée Top 14
scores_J1 = ["20-16","24-19","18-18","13-27","21-17","26-22","12-09"] #à modifier

calendrier2526_J1 = add_scores_for_journee(calendrier2526, 1, scores_J1, inplace=False)
calendrier2526_J1

In [ ]:
from prediction_elo import update_elo_after_journee

In [ ]:
elo_after_J1 = update_elo_after_journee(elo_2526, calendrier2526_J1, J=1, H=H_BEST, s=S_BEST, nu=NU_BEST, )

In [ ]:
elo_after_J1

In [ ]:
#print_probas_journee(elo_after_J1, calendrier2526, journee=2)
#scores_J1 = ["20-16","24-19","18-18","13-27","21-17","26-22","12-09"] #à modifier
#calendrier2526_J2 = add_scores_for_journee(calendrier2526_J1, 2, scores_J2, inplace=False)
#elo_after_J2 = update_elo_after_journee(elo_after_J1, calendrier2526_J2, J=2)
#elo_after_J2

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------